# Content

In this lecture I will introduce:
- The programming language Julia
- Using Jupyter notebooks to run computations and present them
- How to compute the orbit of a (deterministic and random) dynamical system in Julia
- How to compute Birkhoff averages
- Automatic differentiation
- Classical algorithm to approximate Lyapunov exponent (non-rigorous)
- How to make animations to show the Central Limit Theorem in action

# Introduction to Julia and Jupyter

In this notebook I will introduce Julia notation and some of its characteristics, so that it is easy for you to follow the next lectures.
The first thing to stress is that Julia is a language with a different approach than 
1. C, C++, Fortran: if possible Julia infers the type of the variables from the context, so code can be written in a generic way and Julia takes care of the details (most of the time)
2. Python: in Python the code is interpreted, so it runs slowly. Julia does the following, once the types are identified Julia compiles the code. So the first run of a function is slow, but all subsequent runs are fast.

We will see this through examples.

Julia can be used in different forms:
* running the julia shell 
* in Jupyter (or Pluto) notebooks, the environment where we are now
* running scripts in the command line by callin `julia filename.jl`

Due to their ease of use, in these lessons we will focus in using Jupyter notebooks.

Jupyter notebooks consists of cells, of essentially two types: 
- Markdown cells, where text can be written, 
- Code cells where code is written and run. 

I will introduce some of the commands of Jupyter notebooks. 

You can create a new cell above the selected one by pressing __a__ and below the selected one by pressing __b__ . If you select a cell and press __dd__ this will delete the cell. 

You can edit a cell by pressing __ENTER__ when it is selected and you run the code (or compile the Markdown) in a cell by pressing __SHIFT+ENTER__.
Remark that when you select a Markdown cell and press enter, you 

The key __ESC__ will take you to navigation mode, a mode that allows you to move with your arrows between cells.

By default, new cells are Code cells, if you want to turn a code into Markdown, you enter navigation mode, select the cell and press __m__. To change a Markdown cell to code, enter navigation mode, select the cell and press __y__.

In Markdown cells it is possible to write LaTeX code, as 
$$\frac{1}{N-1}\sum_{i=0}^{N-1}\phi(T^i(x_0))$$
and it is possible to use Markdown syntax, that allows to typeset in a similar way as HTML (while being much simpler).
A good reference for Markdown syntax is 
__[DataCamp Markdown tutorial](https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=278443377095&utm_targetid=aud-390929969673:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1001655&gclid=Cj0KCQjwjo2JBhCRARIsAFG667V_O7vei8Tvaa0wRyoIgzbUrKtpHLpPVKBPYlXIMr5p4b5YxTvuXZwaApHUEALw_wcB)__.

#### Exercise 1
Create a cell below this one and convert it to a Markdown cell. Make an unordered list with three items: 
- one equation, my favourite is $e^{2\pi i x}-1=0$
- the name of a theorem, my favourite is Birkhoff Ergodic Theorem
- your country of origin, I'm italian but I now live in Brazil.

It is possible also to make an ordered list:
1. $\pi \approx 3.14159$
2. **Perron-Frobenius theorem**
3. I have lived in Sapporo for one year but now I moved back to Brazil

Now we can start speaking about Julia.
First of all: a line starting with __#__ is a comment.

### Julia: Variable declaration

When working in the Julia shell (called Julia REPL), or in the Jupyter notebook, declaring a variable is really simple, we just write its value with an assignment symbol $=$.

In [ ]:
# This is a code cell

x = 1   # interpreted as an integer
println(x)
println(typeof(x))

In [ ]:
y = 1.0   #interpreted as an Float64 (corresponding to double in C)
println(y)
println(typeof(y))
println(eps(y)) # gives us the machine epsilon

## Julia: Declare a function

It is possible to declare functions using the one-line syntax.

We declare now the function 
$$
f(t, x) = a\cdot x\cdot (1-x)
$$

As in mathematical notation, the variables inside the parentheses are the independent variables (the input)
and the function returns an output.

In [ ]:
dyn(a, x) = a*x*(1-x)

In [ ]:
dyn(4, 0.5)

In [ ]:
a = 3.9
x = 0.5

y = dyn(a, x)

Julia, in general shows as an output the output of the last operation; in our case, the value of $y$.

In the case of more complex functions, it is possible to use the multiline syntax using the keyword `function`

In [ ]:
function dyn_multiline(a, x)
    y = a*x*(1-x)
    return y
end

In [ ]:
dyn_multiline(0.1, 0.01)

In [ ]:
dyn_multiline(-1, 17)

In [ ]:
dyn_multiline(-1, 17)

It is important to remark that we did not specify the type of the arguments or of the output of the function.
What Julia does is that it infers the type of the arguments and of the output when you run the function the first type and compiles a version of the function that works 
for these types. This is called __Just in time compilation__, or __JIT__.
This compiled version of the function is stored and reused later. I will stress this concept many times during this presentation because it is something special to Julia, and that must be taken into account to generate performant code.

Sometimes, reading Julia source code you will see something like this. To write the $\alpha$ in the source code I write `\alpha` and then press `TAB`. 

In [ ]:
function h(x, t = 1; α = 2.0)
    return t-(1+t)*abs(x)^(α)
end

This function has two default input variables, i.e., I can call this function without specifying `t` and `\alpha` explictly.

In [ ]:
h(0)

If needed I can call this function with two arguments, the second argument corresponds to the argument `t` in the argument list. 

In [ ]:
h(0, 0.9)

The parameter `\alpha` is a keyword parameter (we declare them after the `;`) and I can set it without changing  
the default value of `t`.

In [ ]:
h(0.1, α = 3)

In [ ]:
h(0.1, 0.9, α = 5)

### Installing packages and plotting

Before going forward, we would like to install the plotting package for Julia. This can be done in two different ways. Either you go back to the command line, press ] and in the pkg prompt you launch the command "add Plots", or we make visible the Pkg package in our working namespace and use it here. 

Installing a package is something that is needed to be only once for each environment you are using. 
Julia has an environment system, that allows you to have different installed packages and different versions of the packages for different jobs... I will not enter into details here, but it is good to know that we are working in the main julia environment at the moment and there is the possibility of having different environments through the use of the command __activate__.

The following two cells are going to be run only once (in this environment).

In [ ]:
using Pkg # this brings Pkg into our working environment

In [ ]:
Pkg.add("Plots") # this installs Plots in the active environment

### Function compiling and types

In [ ]:
# One line declaration
f(x) = 4x*(1-x)

We will see now the behaviour of the Just in Time compiler of Julia. The first time we will run the function the function is going to be slow, due to compilation time, the second time it is going to be fast because the function is already compiled. Please remark that each time we change the type of the variables Julia is going to compile a new version.

Another thing worth nothing are macros, that begin with __@__. Macros act on the source code of Julia to change it before compilation. The __@time__ macro can be used in front of a function to have a simple benchmark of its behaviour.

We first run and compile the function for Float64 (native floating point numbers).

In [ ]:
x = 0.1
print(typeof(x))
@time f(x)

In [ ]:
@time f(x)

Simply by changing the type of the input, we will compile the function for software multiprecision floating point numbers.

In [ ]:
x = BigFloat(π)/4
@time f(x)

In [ ]:
setprecision(BigFloat, 1024)

As you can see, the output shows the fact that most of the time was spent compiling the function.

In [ ]:
@time f(x)

While up to now we did not speak about the type of variables, under the hood Julia compiles the code, so it infers the type from the context in which we call the function.

We can inspect how Julia infers the types in the code using the macro [@code_warntype](https://docs.julialang.org/en/v1/stdlib/InteractiveUtils/#InteractiveUtils.@code_warntype)

In [ ]:
x = 0.1
@code_warntype f(x)

In [ ]:
x = BigFloat(0.1) 
@code_warntype f(x)

As you can see, in many cases, Julia is able to reuse the same code for different types. This is an extremely useful feature, since it allows us to write simple and reusable code, simplifying the development. I will show you what happens when we try to apply the function to a type for which things don't go so smoothly.

In [ ]:
x = 'A'
f(x)

As you can see, Julia complains saying that it does not know how do the operation
`*` when the left operand is an integer, and the right operand is a character. 
This means that if we Julia to be able to apply the function `f` to a character we need to define the `*` operation for these types. This gives us a script to follow if we want to extend the function `f`.

Once the function is compiled, the computation time radically decreases.

Now, we will bring Plots into the namespace, to plot our function. Plots is a big package, and the first time it runs it is going to be slow. This is one of the main issues with Julia, and the time to first plot is a benchmark for new versions of Julia.

In [ ]:
using Plots # this brings the plot package into our namespace

In [ ]:
@time plot(f, 0, 1)

The second time, again is much faster.

In [ ]:
@time plot(f, 0, 1)

## Computing an orbit

We start now to use the tools we have at our disposal to do some numerical experiments.
The first thing we want to do is to compute an $x, f(x), \ldots, f^{n-1}(x)$ for an initial point $x$.

In [ ]:
function orbit_float(f, x::Float64, n::Int64)
    orb = Array{Float64, 1}(undef, n) #this declares an uninitialized vector
    orb[1] = x
    for i in 2:n
        x = f(x)
        orb[i] = x
    end
    return orb
end

Remark that this time I specified the type of the arguments of the function, i.e., the initial point is a `Float64`,
and the orbit is stored as an array of `Float64`.

We compute an orbit of length $100$ for the point $x=0.1$.

In [ ]:
v100 = orbit_float(f, 0.1, 100)

The following is a Time Series plot of the orbit of the point $0.1$ under the action of the dynamics.

In [ ]:
plot(v100, markershape = :circle, markersize = 3, label = "")

## Generic code

We are interested now in changing the type of the inital point, to a multiprecision floating point number.

In [ ]:
function orbit_mpfr(f, x::BigFloat, n::Int64)
    orb = Array{BigFloat, 1}(undef, n) #this declares an uninitialized vector
    orb[1] = x
    for i in 2:n
        x = f(x)
        orb[i] = x
    end
    return orb
end

If we compare the two functions, they are essentially the same: the only point where we use the type of the initial point `x` is to decide the type of the element of the `orb` vector.
The only thing that we really need is that the type of `x` and of the element of `orb` is the same.
So, we can do the following and let the compiler take care of everything.

In [ ]:
function orbit(f, x, n)
    v = Array{typeof(x), 1}(undef, n) #this declares an uninitialized vector
    v[1] = x
    for i in 2:n
        x = f(x)
        v[i] = x
    end
    return v
end

The code I wrote above is generic; indeed, if I change the type of the x point, the code once with Float64 and run again but with a different type the compiler will compile it for this new type. Here I compute the orbit of the same point, but using higher precision Floating Point numbers, that Julia calls BigFloats and rely on the MPFR library.

In [ ]:
v2000 = orbit(f, 0.1, 2000)

We can run the same function, with multiple precision floating points.

In [ ]:
v2000 = orbit(f, BigFloat(0.1), 2000)

In [ ]:
plot(v2000[1:100], markershape = :circle, markersize = 3, label = "")

Let's take a closer look to how I implemented the code; the important line is the following

```v = Array{typeof(x), 1}(undef, n)```

the Array type is a classical array, as we usually see in C, C++, Fortran.
This is a parametric type and what is written inside __{__ and __}__ are the parameters of this type.
This is going to be an array of elements which have the same type as $x$ of dimension 1.

Inside __(__ and __)__ are the arguments of the function:
- undef tells us that the Array is going to be uninitialized
- n is the length of the array

In [ ]:
z = Array{Int64, 1}(undef, 10) #remark that it is uninitialized, so it contains garbage!!! Be careful!

In [ ]:
z = zeros(Int64, 10) # initializes the array to 0

In [ ]:
z = ones(Int64, 10) # initializes the array to 1

If we want to initalize and array to a specific value, we use the function fill. I will print on screen the documentation thorugh the use of Julia help.

In [ ]:
@doc fill

In [ ]:
fill(3, 10) # this one fills an array with the first argument, the type of the Array is inferred from the value

As you can see, this allows us for a lot of flexibility in writing code. My code above also works for Rational numbers.

In [ ]:
1//7 #this is the rational 1/7

In [ ]:
orbit(f, 1//10, 5) # we don't take an orbit of length 10 because it gives an overflow in Int64

In [ ]:
orbit(f, BigInt(1)//BigInt(10), 10)

If you look back at all the examples in which we computed the orbit, the output contains the type of the elements of the array.

In the last one, the output is of type:
```10-element Vector{Rational{BigInt}}:```

## Vectorizing the functions

Another interesting possibility is the possibility of vectorizing the functions, using the __.__ before the arguments of the functions.

In [ ]:
@doc rand

In [ ]:
x = rand(10) # I take 10 random initial points
orbit.(f, x, 10) # this is going to compute the orbits for these ten points

Suppose now that we want to write a specialized version of the orbit function that returns, instead of an array of arrays a matrix, when we feed it a vector of initial conditions.

To do so, we specialize one of the arguments, i.e., we tell to the compiler that when x is of type 
```Vector{Float64}``` (which is the same as ```Array{Float64, 1}```) we want this specific version of the function orbit to be run.
The compiler is smart, and is going to choose the most specific version of the function.

In [ ]:
function orbit(f, x::Vector{Float64}, n)
    k = length(x)
    v = Array{Float64, 2}(undef, (n, k)) 
    v[1, :] = x
    for i in 2:n
        x = f.(x)
        v[i, :] = x
    end
    return v
end

In [ ]:
@which orbit(f, 0.1, 10)

In [ ]:
@which orbit(f, [0.1, 0.2], 10)

In [ ]:
using Random
orbit(f, rand(MersenneTwister(0), 10), 10)

The compiler is calling different implementations of the function, depending on the type of the arguments.
We will compute the orbit of ten random initial conditions.

The next issue is how we can make the function orbit for a vector more generic, to work with generic types. To do so we will use parametric functions. The function will depend from a parameter, that has to be known at compile time, as the type of the element of our Vector, or the number of its indices (called its dimension).

In [ ]:
function orbit(f, x::Vector{T}, n) where {T} # this where T means that this is a parametric type
    k = length(x)
    v = Array{T, 2}(undef, (n, k)) 
    v[1, :] = x
    for i in 2:n
        x = f.(x)
        v[i, :] = x
    end
    return v
end

Now this works for BigFloat.

In [ ]:
v = orbit(f, 0.5 .+ 0.01*rand(BigFloat, 20), 15); #the semicomma disables the output

In [ ]:
plot(v, label = "")

#### Exercise 4 
Fill in the missing part in the following function, which is a further generalization of orbit, where we can give as an input a matrix or an $N$-index array of initial points and it returns an $N+1$-index array of orbits.
Substitute $\Omega$ for the right expression involving $N$.  

In [ ]:
# this function returns a tuple of (: , :, :, :, ...) that we are going to use to access all the information
# in the multiindex below
fill_colon(N) = ntuple(x-> :, N)

function orbit(f, x::Array{T, N}, n) where {T, N} # now also N is a parameter
    k = size(x)
    v = Array{T, Ω}(undef, (n, k...)) # substitute Ω for an expression with N
    
    l = fill_colon(N) 
    v[1, l...] = x # the ... notation means we are taking the tuple and filling in as argument 2,..., N+1
                    # of the multiindex
    for i in 2:n
        x = f.(x)
        v[i, l...] = x
    end
    return v
end

Now test it here!

In [ ]:
orbit(f, rand(BigFloat, (10, 10)), 20)

### Appendix 1: Floating point numbers
The functions [bitstring](https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring), [nextfloat](https://docs.julialang.org/en/v1/base/numbers/#Base.nextfloat), [prevfloat](https://docs.julialang.org/en/v1/base/numbers/#Base.prevfloat) can be used to explore the structure of the floating point numbers.
The behavior of floating point numbers is delicate, I recommend this beautiful article [What Every Computer Scientist
Should Know About Floating Point
Arithmetic](https://docs.oracle.com/cd/E19957-01/800-7895/800-7895.pdf)

A Float64 number is a sequence of $64$ bits, whose first bit represents the sign $s$, the successive $11$ digits represent a binary number $e$, called the exponent, and the remaining $52$ bits $(d_{52}, \ldots, d_0)$ are the so called mantissa $m$, and represent a binary number of the form 
$$
1+\sum_{i=1}^{52}d_{52-i}\frac{1}{2^{i}}
$$

The number is then represented as 
$$
(-1)^{s}\cdot 2^{e-1023}\cdot m
$$

In [ ]:
y = 1.0
bs = bitstring(y)

In [ ]:
s = parse(Int, "$(bs[1])"; base=2)

We parse the exponent part of the bitstring.

In [ ]:
e = parse(Int, "$(bs[2:12])"; base=2)

We parse now the mantissa part.

In [ ]:
m = 1+parse(Int, "$(bs[13:end])"; base=2)*2^(-52)

We can now reconstruct our number, i.e.,

In [ ]:
(-1)^s*2.0^(e-1023)*m

We observe now what happens when we take the next floating point number.

In [ ]:
bitstring(nextfloat(y))

In [ ]:
nextfloat(y)

Remark that Floating point numbers are few: it is important to recall that numerical computations involve rounding, so to make them mathematically rigorous we need to use tools as [Interval Arithmetic](https://www.amazon.com/Validated-Numerics-Introduction-Rigorous-Computations/dp/069124765X) (this is a link to the book by Prof. Tucker).

In [ ]:
bitstring(prevfloat(y))

In [ ]:
bitstring(prevfloat(-y))

It is important to understand how Julia treats mixed operations, i.e., operations between a Float and an Integer. 

In Julia, types are a fundamental object of the language, it is possible to operate on them and create a hierarchy of types, establish convertion and promotion rules. In the case of an Int64 and a Float64 Point, the two numbers are considered Real Numbers, so Julia converts the Int64 to a Float64.

In [ ]:
x  = 1
@info typeof(x)
y  = 1.0 
@info typeof(y)

z = x + y # Julia automatically promotes the Int64 type to a Float64 type, to make sense of this operation
println(z)
println(typeof(z))


In [ ]:
@code_lowered x+y

In [ ]:
@doc promote

In [ ]:
@less promote(x, y)

In [ ]:
supertypes(Float64)

In [ ]:
supertypes(Int64)

In [ ]:
# we can also create variables with explicit types
x = BigFloat("0.1") # these are MPFR high precision numbers
println(x)

In [ ]:
println(typeof(x))

In [ ]:
println(precision(x)) #get the precision in bit of the mantissa of x

It is important to observe that $0.1$ has no exact representation in base $2$ floating point arithmetic.
If needed, in the REPL, the documentation is obtained by prepending a `@doc` to the name of a function: we are interested in the function setprecision

In [ ]:
@doc setprecision

In [ ]:
setprecision(BigFloat, 1024) # if you run a cell, the output of the last line is given by the cell
y = BigFloat("1.0") # these are MPFR high precision numbers
println(y)
println(typeof(y))
println(precision(y)) #get the precision in bit of the mantissa of x

In [ ]:
z = x+y
precision(z) 
# the output of the last line is given by the cell, here Julia took care of promoting the type 
# to guarantee no precision loss

In [ ]:
x = Float64(π) # to write π, I wrote \pi and pressed the tab key; using this Unicode character
#tells Julia to compute the constant adequate for the type precision
x = BigFloat(π) 

In [ ]:
@code_native 1.0+1.0

In [ ]:
@code_native big"0.1"+big"0.1"

#### Exercise 2:
Create two variables x and y, of type Float64 and type BigFloat respectively, both with value $\pi$.
Compute x-y.

### Appendix 2: Structures and types in Julia
It is possible to declare composite types in Julia. 


In [ ]:
struct WorldCoordinate
    latitude
    longitude
    timezone
end

In [ ]:
RJ = WorldCoordinate(-22.9, -43.19, "GMT-3")

In [ ]:
RJ.latitude # remark that pressing tab you get autocompletion 

In [ ]:
RJ.timezone

Suppose now that we know how to compute a timezone from coordinates (without worrying about the real timezone lines, which are complicated and daylight saving time).

In [ ]:
function timezone(latitude, longitude)  
    val = Int64(floor(longitude/15))
    if val<0
        val = abs(val)
        return "GMT-$val" #the $ symbol makes $val be substituted by val
    elseif val>0
        return "GMT+$val"
    else
        return "GMT-0"
    end
end

We can now define a new constructor for the object.

In [ ]:
WorldCoordinate(a, b) = WorldCoordinate(a, b, timezone(a, b))

In [ ]:
WorldCoordinate(-22, -43) #this returns the object as above, but computes some of its attributes automatically

## Appendix 3: Writing performant code: some notions about variables

We can also specify the type of a variable when we declare it, this helps the compiler infer the right functions and may speed up the code (this requires a version of Julia more recent than 1.8.0).

The reason why this may be necessary is that when using a jupyter notebook (or an interactive programming style), often we define variables in a global scope.
When running the code the compiler need to produce generic code that works for all types, and so the code is slow.

In [ ]:
x_int::Int64 = 32 

I will present three examples, that show how, knowing some information about the types, Julia can speed up a computation in a significant way.
This example is taken from [Blog by Bogumił Kamiński - Why do I use main function in my Julia scripts?](https://bkamins.github.io/julialang/2022/07/15/main.html)


In the first example, we do not give any hint about the types.

In [ ]:
s = 0 
@time for i in 1:10^8
    s+=i
end

In the second example we give informations about the type of the variable that we are using to accumulate the sum.

In [ ]:
s_int::Int64 = 0
@time for i in 1:10^8
    s_int+=i
end

In the third example, we put the computation inside a function, in this case the compiler knows the scope of life of each variable and, given these boundary conditions, can optimize the code.
He check that due to how the function is defined, `s` is and integer and all the operation are integer operations and there are no other possibilities, so he can compile an optimized version of the function (the loop also has a constant length, known at compile time, so he can optimize it a lot).

In [ ]:
function inside_function()
    s = 0
    for i in 1:10^8
        s += i
    end
    return s
end

@time inside_function()

In [ ]:
@code_warntype inside_function()

We can check what is the code generated by the compiler in this case: remark that the compiler has a lot of information at compile time.

In [ ]:
@code_llvm inside_function()

The compiler was so smart that he saw that the return value of the function could be determined at compilation time, so the actual compiled version of the function is nothing else than 
`inside_function() = 5000000050000000`.

If you want to have better informations on the timing of the function, there is a package called BenchmarkTools, that allows to get better estimates on the running time of a function (it runs it many times, then averages the time).

In [ ]:
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

In [ ]:
@btime inside_function()

The functions runs in less than two nanosecond: the compiler was smart enough to identify that the end result was constant and substituted the function by a constant at compile time.

We will come back to how to define functions, but I think it is important to understand that a lot of the ideas behind writing fast code in Julia depend on giving the right hints to the compiler.
In other words, the compiler is really smart, but everybody needs some help sometimes.

What is important to remember is:
**Try to avoid global variables, and if performance is important, write functions**

As a good practice, it is good to encapsulate code in functions.

## Appendix 4: Cobweb plots

We want to implement a simple cobweb plot of the orbit of a point;
i.e., given a point $(x_0, 0)$ we want to plot a line to $(x_0, f(x_0))$,
a line to $(f(x_0), f(x_0))$ and a line to $(f(x_0), f^2(x_0))$ and so on.

In [ ]:
f(x) = 3.83*x*(1-x)

In [ ]:
function orbit(f, x, n)
    v = Array{typeof(x), 1}(undef, n) #this declares an uninitialized vector
    v[1] = x
    for i in 2:n
        x = f(x)
        v[i] = x
    end
    return v
end

In [ ]:
using Plots

In [ ]:
orb = orbit(f, 0.1, 1000)

In [ ]:
x_val = []
y_val = []

x_old = orb[1]
y_old = 0.0

push!(x_val, x_old)
push!(y_val, y_old)
y_old = x_old

for x in orb
    push!(x_val, y_old)
    push!(y_val, y_old)
    push!(x_val, y_old) # this is the point (x_old, f(x_old))
    push!(y_val, x)
    push!(x_val, x) # this is the point (f(x_old), f(x_old))
    push!(y_val, x)
    y_old = x
end

In [ ]:
plot(f, 0, 1, label = "f", color = :blue)

In [ ]:
plot!(x->x, 0, 1, color = :blue, label = "")

In [ ]:
plot!(x_val, y_val, color = :red, label = "")

## Appendix 5: Bifurcation diagram

We will plot now the bifurcation diagram for the quadratic family.

In [ ]:
function orbit_with_transient(f, x, n; transient = 9*n)
    
    for i in 1:transient
        x = f(x)
    end
    
    
    v = Array{typeof(x), 1}(undef, n) #this declares an uninitialized vector
    v[1] = x
    for i in 2:n
        x = f(x)
        v[i] = x
    end
    return v
end

In [ ]:
dyn(a, x) = a*x*(1-x)

In [ ]:
parameters = 0:0.01:4

In [ ]:
plt = plot()

for a in parameters
    orb = orbit_with_transient(x->dyn(a, x), 0.5, 100)
    plt = scatter!(plt, fill(a, 1000), orb, label = "", markersize = 0.1)
end
xlims!(plt, 0, 4)
ylims!(plt, 0, 1)